In [3]:
from smooth import is_smooth
from random import randint, seed, random, Random

In [2]:
_rng = Random()

def rand_smooth(y, B):
    x = _rng.randrange(1, B + 1)
    while not is_smooth(x, y):
        x = _rng.randrange(1, B + 1)
    return x

In [4]:
rand_smooth(2139321, 4219312009312)

2404785386858

In [6]:
def legendre(a, p):
    a_mod = a % p
    if a_mod in {0,1}:
        return a_mod
    if a_mod == p - 1:
        return 1 if p % 4 == 1 else -1
    if a_mod == 2:
        mod_temp = p % 8
        return 1 if mod_temp in {1, 7} else -1
    exp = (p - 1) // 2
    return pow(a, exp, p)